In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-17 18:26:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.12MB/s    in 0.2s    

2022-03-17 18:26:58 (6.12 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://databootcamp-bucket-llebl095.s3.amazonaws.com/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R33UPQQUZQEM8|          5|            3|          3|   N|                Y|
|R3IKTNQQPD9662|          5|            0|          0|   N|                Y|
|R3U27V5QMCP27T|          5|            1|          1|   N|                Y|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|                Y|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|                Y|
|R1N1KHBRR4ZTX3|          5|            0|          0|   N|                Y|
|R3OM9S0TCBP38K|          5|            0|          0|   N|                Y|
|R1W4S949ZRCTBW|          5|            0|          0|   N|                Y|
|R18JL1NNQAZFV2|          5|            0|          0|   N|                Y|
|R1LP6PR06OPYUX|          4|            0|          0|   N|     

In [7]:
#step two
#Filter the data and create a new DataFrame or table to retrieve all the rows ,where the total_votes count is equal to or greater than 20
greater_vine_vote_df = vine_df.filter("total_votes>=20")
greater_vine_vote_df .show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R12Q1NO1HI9PP4|          1|            4|         86|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
| R6TNONSNFM82I|          1|            0|         28|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
| RUNB838XKMB59|          1|            8|         23|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
|R2WFUSIPJ3X0FR|          1|            9|         53|   N|                N|
| RONWXF0J85CRR|          5|           42|         50|   N|     

In [9]:
# Create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_vine_votes = greater_vine_vote_df.filter("helpful_votes/total_votes >= 0.5")
helpful_vine_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
| RONWXF0J85CRR|          5|           42|         50|   N|                N|
|R2QQJIJZIUSQBH|          1|           16|         20|   N|                N|
|R3O7T74HRIXBE5|          5|           53|         55|   N|                Y|
| RSU37FK0W4DPE|          4|           26|         28|   N|                N|
|R22H100N6EUD0Y|          5|           30|         32|   N|     

In [14]:
review_vine_vote_df = helpful_vine_votes.filter(helpful_vine_votes.vine == 'Y')
review_vine_vote_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PBQ665MBL8CP|          3|           66|         72|   Y|                N|
|R3EMD30K7MNOZ0|          2|           20|         25|   Y|                N|
|R2GF3ZLO2B0Y71|          5|          236|        243|   Y|                N|
|R1F8T2Z7U3N6HL|          3|           11|         20|   Y|                N|
| R5QTOT1GVR8RG|          4|           16|         20|   Y|                N|
|R1KAQEESF0NUGV|          4|           27|         29|   Y|                N|
|R1621B9AMVWI25|          2|           27|         29|   Y|                N|
|R3FVGQVNBAZL8E|          3|           37|         44|   Y|                N|
|R26ATTA765WHVX|          5|           21|         23|   Y|                N|
| RMZYPFKES1ZK7|          4|           29|         33|   Y|     

In [15]:
review_vine_vote_2_df = helpful_vine_votes.filter(helpful_vine_votes.vine == 'N')
review_vine_vote_2_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
| RONWXF0J85CRR|          5|           42|         50|   N|                N|
|R2QQJIJZIUSQBH|          1|           16|         20|   N|                N|
|R3O7T74HRIXBE5|          5|           53|         55|   N|                Y|
| RSU37FK0W4DPE|          4|           26|         28|   N|                N|
|R22H100N6EUD0Y|          5|           30|         32|   N|     

In [16]:

#Determine the total number of paid reviews
total_paid_reviews_df = review_vine_vote_df.count()
total_paid_reviews_df

49

In [17]:
# the number of 5-star reviews
five_star_review_df = review_vine_vote_df.filter(review_vine_vote_df.star_rating == '5').count()
five_star_review_df

9

In [18]:
# the percentage of 5-star reviews for the two types of review
percent_fivestar_review_df = ((five_star_review_df / total_paid_reviews_df) * 100)
percent_fivestar_review_df

18.367346938775512

In [19]:
# Number of unpaid reviews
total_unpaid_reviews_df = review_vine_vote_2_df.count()
total_unpaid_reviews_df

151400

In [21]:
# the number of 5-star reviews
unpaid_five_star_review_df = review_vine_vote_2_df.filter(review_vine_vote_2_df.star_rating == '5').count()
unpaid_five_star_review_df

78061

In [22]:
u_percent_fivestar_review_df = ((unpaid_five_star_review_df / total_unpaid_reviews_df) * 100)
u_percent_fivestar_review_df

51.55944517833554